## Testing RAG with RAGAs

### Setup LLM

In [1]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

my_llm = ChatOllama(
    base_url="http://localhost:11434",
    model="llama3.1:8b",
    temperature=0.6,
    max_tokens=300
)

openai_llm = ChatOpenAI(
    model="gpt-4o"
)

load_dotenv('./../.env')

True

### Setup RAG & Retriever

In [2]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma(
    persist_directory='./../Section_10_IntroToHuggingFace/chroma_db_hf',
    embedding_function=embeddings
)

# retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {'k': 5})

# response = retriever.invoke("Full name of Parveen ?")

# for doc in response:
#     print(doc.metadata["source"])

/Users/parveenshaikh/Study/AI/Courses/LangChain/myenvLangchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup RetrievalQA

In [3]:
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever(search_kwargs={'k':3})

qa_chain = RetrievalQA.from_chain_type(llm=my_llm, retriever=retriever)

query = "What is full name of Dhanesh ?"

response = qa_chain.invoke(query)
retrieved_docs = retriever.get_relevant_documents(query)

response['result']

/var/folders/xl/2h_vnm9s4dl0fnn4x8_mg4q40000gn/T/ipykernel_60642/1667655701.py:10: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


'The full name of the person in question is GUJRATHI DHANESH NARESH.'

### Creating MultiShotSample Dataset for RAGAs 📈📉

In [4]:
# Reference for below implementation - 
# https://docs.ragas.io/en/stable/concepts/components/eval_dataset/#creating-an-evaluation-dataset-from-singleturnsamples

questions = [
    "What is full name of Dhanesh ?",
    "What is full name of Parveen ?",
    "Which organisation applied for the employment pass of Dhanesh in 2019 ?",
    "Which organisation applied for the employment pass of Parveen in 2024 ?",
    "Which organisation applied for the employment pass of Dhanesh in 2020 ?"
]

dataset = []

all_docs = vector_store.get(include=["documents"])["documents"]

for question, doc in zip(questions, all_docs):
    relevant_docs = [doc.page_content for doc in retriever.invoke(question)]
    response = qa_chain.invoke(question)
    
    dataset.append({
        "user_input": question,
        "retrieved_contexts": relevant_docs,
        "response": response['result'],
        "reference": doc
    })

dataset

[{'user_input': 'What is full name of Dhanesh ?',
  'retrieved_contexts': ["Name: GUJRATHI DHANESH NARESH\nFIN: G3872158K\nDate of Application: 01 JUL 2020\nDate of Birth: 24 DEC 1992\nNationality: INDIAN\nEducational details:\nCountry of Awarding Body/Institution/University: INDIA\nName of Awarding Body/Institution/University: SAVITRIBAI PHULE PUNE UNIVERSITY\nMain Campus or Affiliating College Attended: SAVITRIBAI PHULE PUNE UNIVERSITY - AFFILIATING \nCOLLEGES\nQualification: BACHELOR'S DEGREE\nFaculty: ENGINEERING\nMode of study: FULL-TIME\nPeriod of study: 01 AUG 2010 to 01 MAY 2014\nName of Employer/Sponsor: NITYO INFOTECH SERVICES PTE. LTD.",
   "FOREIGN EMPLOYEE'S NAME\nGUJRATHI DHANESH NARESH\nDATE OF BIRTH\n24 DEC 1992\nSEX\nMALE\nNATIONALITY\nINDIAN\nPASSPORT NO\nM1525989\nFIN\nG3872158K\nDATE OF APPLICATION\n01 JUL 2020\nOCCUPATION\nTECHNICAL SUPPORT ENGINEER\n● You must comply with the conditions under the Employment of Foreign Manpower Act and conditions of the \nEmploymen

### Evaluation of RAG data by RAGAs

In [8]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import Faithfulness, LLMContextRecall, ContextPrecision, AnswerRelevancy

load_dotenv(override=True)
load_dotenv('./../.env')

# eval_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
eval_llm = LangchainLLMWrapper(ChatOpenAI(model=""))

# openai_llm.invoke("Singapore avergae salary for IT jobs?")

output = evaluate(dataset=evaluation_dataset, 
                  metrics=[LLMContextRecall(), Faithfulness(), ContextPrecision(), AnswerRelevancy()], 
                  llm=eval_llm)

output

Evaluating: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]


{'context_recall': 0.3386, 'faithfulness': 0.9500, 'context_precision': 0.4167, 'answer_relevancy': 0.8890}

In [19]:
output.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,context_precision,answer_relevancy
0,What is full name of Dhanesh ?,[Name: GUJRATHI DHANESH NARESH\nFIN: G3872158K...,The full name of the individual in question is...,AIPA - Employment Pass G3872158K / 01 Jul 2020...,0.222222,1.00,1.000000,0.836376
1,What is full name of Parveen ?,[Name SHAIKH PARVEEN \nDAUD \nDate of Birth 16...,The full name of the person in question is SHA...,next steps you need to take so that an Employm...,0.250000,1.00,0.583333,0.840222
2,Which organisation applied for the employment ...,[AIPA - Employment Pass G3872158K / 01 Jul 202...,"According to the provided context, NTT DATA SI...",FOREIGN EMPLOYEE'S NAME\nGUJRATHI DHANESH NARE...,0.363636,1.00,0.500000,0.904050
3,Which organisation applied for the employment ...,[particulars to submit your request and \nthe ...,"According to the context, it was actually ""NIT...","Pass, prosecute you and withdraw your permissi...",0.285714,0.75,0.000000,0.918458
4,Which organisation applied for the employment ...,[AIPA - Employment Pass G3872158K / 01 Jul 202...,Nityo Infotech Services PTE. LTD. applied for ...,IPA - Employment Pass G3872158K / 01 Jul 2020\...,0.571429,1.00,0.000000,0.946082
